# Using LLMs

This Jupyter Notebook demonstrates the use of various Large Language Model (LLM) providers and tools for natural language processing tasks. It showcases how to interact with LLMs using the `crewai` library (which is used by Agentics), parse structured outputs with Pydantic models, and explore available LLM connections. The notebook provides practical examples for calling different LLMs, formatting responses, and integrating with external providers such as OpenAI and IBM WatsonX.

In [ ]:
! uv pip install agentics-py


import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive
    drive.mount("/content/drive")
    load_dotenv("/content/drive/MyDrive/.env")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 15ms
In Colab: False


## Connect to your own LLM provider
Agentics uses CrewAI wrappers for main LLM providers. You can initialize your LLM as follows.
[find out more](https://docs.crewai.com/en/concepts/llms)

In [2]:
from crewai import LLM

# pick a provider (openai, anthropic, groq, etc.) - see crewai docs for details
llm = LLM(model="gemini/gemini-2.0-flash",
    temperature=0.7,    # Adjust based on task
    max_tokens=4096,    # Set based on output needs
    timeout=300)        # Longer timeout for complex tasks

print(llm)

## Perform Simple LLM call

Once an LLM is instatiated, you can perform LLM calls

In [3]:
print(llm.call("where is the Eiffel Tower?"))

The Eiffel Tower is located in **Paris, France**.



## Perform Structured Call

LLMs can generate structured objects given a pydantic schema if you instantiate them accordingly

In [4]:
from pydantic import BaseModel
class Answer(BaseModel):
    short_answer: str
    detailed_answer: str
    confidence: float

struct_llm = LLM(model="gemini/gemini-2.0-flash", response_format=Answer)

print(struct_llm.call("Why is the sky blue?"))


{
  "short_answer": "The sky is blue due to a phenomenon called Rayleigh scattering.",
  "detailed_answer": "Sunlight is made up of all the colors of the rainbow. When sunlight enters the Earth's atmosphere, it is scattered in different directions by air molecules and other small particles. Blue and violet light have shorter wavelengths and are scattered more than other colors. Because our eyes are more sensitive to blue than violet, we perceive the sky as blue. At sunrise and sunset, the sunlight travels through more of the atmosphere, so more of the blue light is scattered away, leaving the longer wavelengths like red and orange to be more visible.",
  "confidence": 0.95
}


## You can also use structured decoding for information extraction

In [5]:
import requests
class Person(BaseModel):
    name: str
    city: str
    occupation: str

class Book(BaseModel):
    title: str
    publisher: str
    year: int
    author: Person

class InformationExtraction(BaseModel):
    books: list[Book]
    people: list[Person]
    
person_llm = LLM(model="gemini/gemini-2.0-flash", response_format=InformationExtraction)
print(person_llm.call(requests.get("https://iep.utm.edu/wittgens/").text))



{
  "books": [
    {
      "title": "Tractatus Logico-Philosophicus",
      "publisher": "Routledge and Kegan Paul",
      "year": 1961,
      "author": {
        "name": "Ludwig Wittgenstein",
        "city": "Vienna",
        "occupation": "Philosopher"
      }
    },
    {
      "title": "The Blue and Brown Books",
      "publisher": "Basil Blackwell",
      "year": 1969,
      "author": {
        "name": "Ludwig Wittgenstein",
        "city": "Vienna",
        "occupation": "Philosopher"
      }
    },
    {
      "title": "Philosophical Investigations",
      "publisher": "Basil Blackwell",
      "year": 1963,
      "author": {
        "name": "Ludwig Wittgenstein",
        "city": "Vienna",
        "occupation": "Philosopher"
      }
    },
    {
      "title": "On Certainty",
      "publisher": "Basil Blackwell",
      "year": 1979,
      "author": {
        "name": "Ludwig Wittgenstein",
        "city": "Vienna",
        "occupation": "Philosopher"
      }
    },
    {
      "t

## Using Agentics Predefined LLMs

Agentics has the following LLM handles: watsonx_llm, openai_llm, gemini_llm
You can directly import and use them as defaults

In [6]:
from agentics.core.llm_connections import available_llms, get_llm_provider
print(available_llms)

llm=get_llm_provider() ## get the default LLM provider from the available ones
print(llm.call("Where is Rome?"))

llm=get_llm_provider("gemini") ## get a specific LLM provider from the available ones,change "gemini" with "openai", "watsonx", etc. as needed
print(llm.call("Where is Rome?"))

2025-09-11 11:17:27.220 | DEBUG    | agentics.core.llm_connections:<module>:121 - AGENTICS is connecting to the following LLM API providers:
2025-09-11 11:17:27.221 | DEBUG    | agentics.core.llm_connections:<module>:124 - 0 - WatsonX
2025-09-11 11:17:27.221 | DEBUG    | agentics.core.llm_connections:<module>:129 - 1 - Gemini
2025-09-11 11:17:27.221 | DEBUG    | agentics.core.llm_connections:<module>:133 - 2 - OpenAI
2025-09-11 11:17:27.221 | DEBUG    | agentics.core.llm_connections:<module>:135 - Please add API keys in .env file to add or disconnect providers.
2025-09-11 11:17:27.228 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-11 11:17:27.228 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'
2025-09-11 11:17:27.229 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. U

{'watsonx': <crewai.llm.LLM object at 0x157c16b40>, 'gemini': <crewai.llm.LLM object at 0x168a51be0>, 'openai': <crewai.llm.LLM object at 0x168a51d60>}


2025-09-11 11:17:32.066 | DEBUG    | agentics.core.llm_connections:get_llm_provider:42 - Using specified LLM provider: gemini


Rome is the capital city of Italy, located in the central part of the country, within the Lazio region. It is situated on the Tiber River and is home to many historic landmarks, including the Colosseum and the Vatican City.
Rome is the capital city of **Italy**. It's located in the central-western part of the Italian Peninsula, along the Tiber River.



## Homework

Play with different data models on your favourite domain (e.g. stocks) providing relevant data. Enjoy tructured decoding magic.